In [1]:
!pip install -q opencv-python

In [2]:
from qonnx.util.cleanup import cleanup
from finn.util.visualization import showSrc, showInNetron
import numpy as np
import cv2

# View Original Model in Netron

In [3]:
#model_ori = './BED_classifier__med_comp__NO_Identity__BIPOLAR__FLOOR__QONNX.onnx'
#model_ori = './BED_classifier__med_comp__NO_Identity__FLOOR__QONNX.onnx'
model_ori = './pretrained_qonnx.onnx'

In [4]:
showInNetron(model_ori)

Serving './pretrained_qonnx.onnx' at http://0.0.0.0:8083


# Clean Model with QONNX

In [5]:
model_cleaned = './pretrained_qonnx_cleaned.onnx'

In [6]:
cleanup(model_ori, out_file=model_cleaned)

In [7]:
showInNetron(model_cleaned)

Stopping http://0.0.0.0:8083
Serving './pretrained_qonnx_cleaned.onnx' at http://0.0.0.0:8083


# Import Cleaned Model to FINN

In [8]:
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as oxe

In [9]:
model_cleaned_onnx = ModelWrapper(model_cleaned)

# Forward Pass

In [10]:
dummy_in = np.random.randint(low = 0, high = 255+1, size = (1, 3, 224, 224)) 
dummy_in = (dummy_in / 256.).astype(np.float32)
#dummy_in

In [11]:
#import matplotlib.pyplot as plt

In [12]:
#plt.imshow(dummy_in[0].transpose(1, 2, 0))

## Cleaned

In [13]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(model_cleaned_onnx, input_dict)
produced_qonnx = output_dict[list(output_dict.keys())[0]]
produced_qonnx

array([[-0.7223735, -2.6005445]], dtype=float32)

## Check with Image

In [58]:
img_file = './WEB10495.jpg' # Smoke and Fire
#img_file = './WEB10980.jpg' # Only Smoke
#img_file = './WEB10031.jpg' # Empty
img = cv2.imread(img_file)
img = cv2.imread(img_file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
img = cv2.resize(img, (224,224), interpolation = cv2.INTER_LINEAR) 
img = img / 256.
img = np.expand_dims(img, axis=0)
img = np.transpose(img, (0, 3, 1, 2))
img = img.astype(np.float32)

In [15]:
print(f'Image shape: {img.shape}')
print(f'Data type: {img.dtype}')

Image shape: (1, 3, 224, 224)
Data type: float32


In [16]:
input_dict = {"global_in": img}
output_dict = oxe.execute_onnx(model_cleaned_onnx, input_dict)
produced_qonnx = output_dict[list(output_dict.keys())[0]]
produced_qonnx

array([[2.0226457, 3.0339684]], dtype=float32)

# Convert QONNX to FINN-ONNX

In [18]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

In [26]:
model_finn_onnx = ModelWrapper(model_cleaned)

model_finn_onnx = model_finn_onnx.transform(ConvertQONNXtoFINN())

In [27]:
model_finn_path = './pretrained_qonnx_cleaned_FINN.onnx'

In [28]:
model_finn_onnx.save(model_finn_path)

In [29]:
showInNetron(model_finn_path)

Stopping http://0.0.0.0:8083
Serving './pretrained_qonnx_cleaned_FINN.onnx' at http://0.0.0.0:8083


In [30]:
model_finn_onnx = ModelWrapper(model_finn_path)
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(model_finn_onnx, input_dict)
produced_finn = output_dict[list(output_dict.keys())[0]]

produced_finn

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor wnIQch can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(


array([[0., 0.]], dtype=float32)

In [38]:
for node in model_finn_onnx.graph.node:
    if node.output[0] == 'wnIQch':
        print(node)
    # print(node.output)
    # break

input: "global_in"
input: "vzTQ2h"
output: "wnIQch"
op_type: "MultiThreshold"
attribute {
  name: "out_dtype"
  s: "INT8"
  type: STRING
}
domain: "qonnx.custom_op.general"



In [39]:
np.isclose(produced_qonnx, produced_finn).all()

False

### Exceute only certain nodes

In [65]:
out_dict_finn = oxe.execute_onnx(
    model_finn_onnx, 
    input_dict, 
    return_full_exec_context=True,
    # start_node=model_finn_onnx.graph.node[0],
    # end_node=model_finn_onnx.graph.node[1]
)

In [67]:
out_dict_finn['TrSs03']

array([[[[ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         ...,
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.]],

        [[ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         ...,
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.]],

        [[ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         ...,
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.]],

        ...,

        [[ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.,  7., ...,  7.,  7.,  7.],
         [ 7.,  7.

In [68]:
out_dict_cleaned = oxe.execute_onnx(
    model_cleaned_onnx,
    input_dict, 
    return_full_exec_context=True,
    # start_node=model_cleaned_onnx.graph.node[0],
    # end_node=model_cleaned_onnx.graph.node[1]
)

In [70]:
#print(type(out_dict_cleaned['Quant_29_out0']))
out_dict_cleaned['Conv_0_out0']

array([[[[ 4.07477587e-01,  9.05506238e-02,  1.03486389e-01, ...,
           2.26376448e-02,  2.26376448e-02, -3.88073891e-01],
         [ 4.72156525e-01,  1.87569022e-01,  1.90803021e-01, ...,
           2.26376448e-02,  2.26376448e-02, -3.88073891e-01],
         [ 4.85092312e-01,  1.87568933e-01,  1.81101292e-01, ...,
           2.26376448e-02,  2.26376448e-02, -3.88073891e-01],
         ...,
         [ 2.10206747e-01, -9.37845483e-02, -1.55229628e-01, ...,
           1.71399236e-01,  1.71399266e-01,  7.11469054e-02],
         [ 2.03738749e-01, -1.26124114e-01, -1.29358053e-01, ...,
           2.03738719e-01,  1.87569052e-01,  4.52752784e-02],
         [ 1.42293692e-01, -4.52753603e-02, -5.17432317e-02, ...,
           1.87568963e-01,  1.64931387e-01,  2.58716159e-02]],

        [[-2.84587502e-01,  1.03486404e-01,  1.06720328e-01, ...,
           7.95551479e-01,  7.95551479e-01,  1.20626295e+00],
         [-2.32844293e-01,  2.58715898e-01,  2.68417716e-01, ...,
           1.61697447e

In [52]:
np.isclose(out_dict_cleaned['Quant_29_out0'], out_dict_finn['Quant_29_out0']).all()

True

In [56]:
def compare_all_nodes_out(model1, model2, input_dict):

    out_1 = oxe.execute_onnx(
        model1, 
        input_dict, 
        return_full_exec_context=True
    )

    out_2 = oxe.execute_onnx(
        model2, 
        input_dict, 
        return_full_exec_context=True
    )
    
    for key1 in out_1.keys():
        for key2 in out_2.keys():
            if key1 == key2:
                print(key1)
                print(f'\tOut is equal? {np.isclose(out_1[key1], out_2[key2]).all()}')

In [62]:
input_dict = {"global_in": img}
#input_dict = {"global_in": dummy_in}

In [63]:
compare_all_nodes_out(
    model1 = model_finn_onnx,
    model2 = model_cleaned_onnx,
    input_dict = input_dict
)

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor wnIQch can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(


global_in
	Out is equal? True
global_out
	Out is equal? False
Quant_0_out0
	Out is equal? True
Quant_29_out0
	Out is equal? False
Quant_30_out0
	Out is equal? False
Quant_31_out0
	Out is equal? False
Quant_32_out0
	Out is equal? False
Quant_33_out0
	Out is equal? False
Quant_34_out0
	Out is equal? False
Quant_35_out0
	Out is equal? False
Quant_36_out0
	Out is equal? False
Quant_37_out0
	Out is equal? False
Quant_38_out0
	Out is equal? False
Quant_39_out0
	Out is equal? False
Quant_40_out0
	Out is equal? False
Trunc_0_out0
	Out is equal? False
Quant_41_out0
	Out is equal? False
Conv_0_out0
	Out is equal? False
MaxPool_0_out0
	Out is equal? False
Conv_1_out0
	Out is equal? False
MaxPool_1_out0
	Out is equal? False
Conv_2_out0
	Out is equal? False
Conv_3_out0
	Out is equal? False
Conv_4_out0
	Out is equal? False
Conv_5_out0
	Out is equal? False
MaxPool_2_out0
	Out is equal? False
Conv_6_out0
	Out is equal? False
Conv_7_out0
	Out is equal? False
Conv_8_out0
	Out is equal? False
Conv_9_out0

## Check with Image

In [31]:
img_file = './WEB10495.jpg' # Smoke and Fire
#img_file = './WEB10980.jpg' # Only Smoke
#img_file = './WEB10031.jpg' # Empty
img = cv2.imread(img_file)
img = cv2.imread(img_file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
img = cv2.resize(img, (224,224), interpolation = cv2.INTER_LINEAR) 
img = img / 256.
img = np.expand_dims(img, axis=0)
img = np.transpose(img, (0, 3, 1, 2))
img = img.astype(np.float32)

In [32]:
input_dict = {"global_in": img}

In [33]:
output_dict = oxe.execute_onnx(model, input_dict)
produced_finn = output_dict[list(output_dict.keys())[0]]

produced_finn

array([[0., 0.]], dtype=float32)

In [92]:
#model.__dict__

In [93]:
model.graph.node[1]

input: "HNK4HC"
input: "KoxANs"
output: "nbVGqX"
op_type: "Mul"

In [68]:
from finn.core.onnx_exec import execute_onnx

In [108]:
execute_onnx(
    model, 
    input_dict, 
    return_full_exec_context=False,
    start_node=model.graph.node[0],
    end_node=model.graph.node[10]
)

{'global_out': array([[0., 0.]], dtype=float32)}

In [109]:
i = 0
for node in model_cleaned_onnx.graph.node:
    if node.name == "Conv_0":
        print(f'Conv_0 is node: {i}')
        #print(node)
    elif node.name == "Relu_0":
        print(f'Relu_0 is node: {i}')
        #print(node)
    elif node.name == "MaxPool_0":
        print(f'MaxPool_0 is node: {i}')
        #print(node)
    elif node.name == "Gemm_1":
        print(f'Gemm_1 is node: {i}')
        #print(node)
    i += 1

Conv_0 is node: 33
Relu_0 is node: 34
MaxPool_0 is node: 36
Gemm_1 is node: 84


In [110]:
model_cleaned_onnx.graph.node[84]

input: "Quant_45_out0"
input: "Quant_31_out0"
input: "Quant_32_out0"
output: "global_out"
name: "Gemm_1"
op_type: "Gemm"
attribute {
  name: "alpha"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "beta"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "transB"
  i: 1
  type: INT
}

In [111]:
execute_onnx(
    model_cleaned_onnx, 
    input_dict, 
    return_full_exec_context=True,
    start_node=model_cleaned_onnx.graph.node[33],
    end_node=model_cleaned_onnx.graph.node[84]
)

{'global_in': array([[[[0.3125    , 0.3125    , 0.32421875, ..., 0.9921875 ,
           0.9921875 , 0.9921875 ],
          [0.30859375, 0.3125    , 0.32421875, ..., 0.9921875 ,
           0.9921875 , 0.9921875 ],
          [0.3125    , 0.31640625, 0.32421875, ..., 0.9921875 ,
           0.9921875 , 0.9921875 ],
          ...,
          [0.23046875, 0.25      , 0.23828125, ..., 0.2265625 ,
           0.2109375 , 0.234375  ],
          [0.23828125, 0.2421875 , 0.2265625 , ..., 0.23046875,
           0.23828125, 0.25390625],
          [0.22265625, 0.2265625 , 0.234375  , ..., 0.234375  ,
           0.2421875 , 0.23046875]],
 
         [[0.2734375 , 0.27734375, 0.2890625 , ..., 0.9921875 ,
           0.9921875 , 0.9921875 ],
          [0.26953125, 0.27734375, 0.2890625 , ..., 0.9921875 ,
           0.9921875 , 0.9921875 ],
          [0.2734375 , 0.28125   , 0.2890625 , ..., 0.9921875 ,
           0.9921875 , 0.9921875 ],
          ...,
          [0.265625  , 0.29296875, 0.28515625, ..., 0.